In [106]:
import urllib.request as urllib2
import sys
import common

TARGET = 'http://crypto-class.appspot.com/po?er='
#--------------------------------------------------------------
# padding oracle
#--------------------------------------------------------------
class PaddingOracle(object):
    def query(self, q):
        target = TARGET + urllib2.quote(q)    # Create query URL
        req = urllib2.Request(target)         # Send HTTP request to server
        try:
            f = urllib2.urlopen(req)          # Wait for response
            return True
        except urllib2.HTTPError as e:          
            print("We got: %d" % e.code)       # Print response code
            if e.code == 404:
                return True # good padding
            return False # bad padding

if __name__ == "__main__":
    po = PaddingOracle()
    po.query("f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4")       # Issue HTTP query with the given argument

http://crypto-class.appspot.com/po?er=f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4


In [107]:
BLOCK_SIZE = 16

In [113]:
ctext = bytes.fromhex("f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4")
ctext_blocks = [bytes(ctext[i:i+BLOCK_SIZE]) for i in range(0, len(ctext), BLOCK_SIZE)]
num_blocks = len(ctext_blocks)
decrypted = []
blocks_decrypted = 0

for i in reversed(range(len(ctext) - BLOCK_SIZE)):
    found_byte = False
    for guess in range(256):
        padding_len = 1 + len(decrypted) % 16
        # skip this trivial solution
        if guess == 1 and padding_len == 1:
            continue
        decrypted_blocks = len(decrypted)//16
        padded_block_idx = num_blocks - 2 - decrypted_blocks
        padded_block = ctext_blocks[padded_block_idx][:BLOCK_SIZE - padding_len] + \
            common.xor(ctext_blocks[padded_block_idx][BLOCK_SIZE - padding_len:], \
                       bytes([b ^ padding_len for b in [guess] + decrypted[:padding_len - 1]]))
        argument = ''.join([block.hex() for block in [padded_block] + [ctext_blocks[padded_block_idx + 1]]])
        if po.query(argument):
            found_byte = True
            decrypted = [guess] + decrypted
            print(decrypted)
            break
    # This should be unreachable
    if found_byte == False:
        print("No guess worked!!")
        break
print(decrypted)

4a61044426fb515dad3f21f18aa577c1bdf302936266926ff37dbf7035d5eeb4
64
0
http://crypto-class.appspot.com/po?er=4a61044426fb515dad3f21f18aa577c1bdf302936266926ff37dbf7035d5eeb4
We got: 403
4a61044426fb515dad3f21f18aa577c3bdf302936266926ff37dbf7035d5eeb4
64
2
http://crypto-class.appspot.com/po?er=4a61044426fb515dad3f21f18aa577c3bdf302936266926ff37dbf7035d5eeb4
We got: 403
4a61044426fb515dad3f21f18aa577c2bdf302936266926ff37dbf7035d5eeb4
64
3
http://crypto-class.appspot.com/po?er=4a61044426fb515dad3f21f18aa577c2bdf302936266926ff37dbf7035d5eeb4
We got: 403
4a61044426fb515dad3f21f18aa577c5bdf302936266926ff37dbf7035d5eeb4
64
4
http://crypto-class.appspot.com/po?er=4a61044426fb515dad3f21f18aa577c5bdf302936266926ff37dbf7035d5eeb4
We got: 403
4a61044426fb515dad3f21f18aa577c4bdf302936266926ff37dbf7035d5eeb4
64
5
http://crypto-class.appspot.com/po?er=4a61044426fb515dad3f21f18aa577c4bdf302936266926ff37dbf7035d5eeb4
We got: 403
4a61044426fb515dad3f21f18aa577c7bdf302936266926ff37dbf7035d5eeb4
64
6
http:

In [114]:
bytes([84, 104, 101, 32, 77, 97, 103, 105, 99, 32, 87, 111, 114, 100, 115, 32, 97, 114, 101, 32, 83, 113, 117, 101, 97, 109, 105, 115, 104, 32, 79, 115, 115, 105, 102, 114, 97, 103, 101, 9, 9, 9, 9, 9, 9, 9, 9, 9])

b'The Magic Words are Squeamish Ossifrage\t\t\t\t\t\t\t\t\t'